In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import urllib.request


print("=" *50)
print("    지마켓 Best Seller 상품 정보 추출하기")
print("=" *50)
print("\n")

title='지마켓'
url='http://corners.gmarket.co.kr/Bestsellers'

cnt = int(input("   1.크롤링 할 건수는 몇건입니까?: "))
f_dir = input("   2.결과 파일을 저장할 폴더명만 쓰세요(예:C:\py_temp\): ")


now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+title+'-')
os.chdir(f_dir+s+'-'+title+'-')

ft_dir=f_dir+s+'-'+title+'-'
ft_name=f_dir+s+'-'+title+'-'+'\\'+s+'-'+title+'-'+'.txt'
fc_name=f_dir+s+'-'+title+'-'+'\\'+s+'-'+title+'-'+'.csv'
fx_name=f_dir+s+'-'+title+'-'+'\\'+s+'-'+title+'-'+'.xls'

#크롬 드라이버 실행
s_time = time.time( )

path = "C:\py_temp\chromedriver_win32\chromedriver.exe" # 컴퓨터 환경에 따라 변경해야함?
driver = webdriver.Chrome(path)
    
driver.get(url)
time.sleep(5)

#스크롤 다운
def scroll_down(driver):
      
    driver.execute_script("window.scrollBy(0,17550);")
    time.sleep(1)

scroll_down(driver)

bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

img_src2=[]   # 이미지 URL저장변수
file_no = 0

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#reple_result = soup.select('.best-list')
reple_result = soup.select('.best-list')[1]
slist = reple_result.select('ul > li')
#print(reple_result)

if cnt > 200 :
    cnt = 200
    print("검색 건수는 1건 - 최대 200 건까지만 가능합니다")

ranking2=[]
title3=[]
price2=[]
sale_price2=[]
rate_price2=[]

count = 0
    
# 이미지 저장용 폴더생성
img_dir = ft_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

for li in slist :
    #이미지
    try :
        photo = li.find('div','thumb').find('img')['src']
    except AttributeError :
        continue
    file_no += 1
    urllib.request.urlretrieve(photo,str(file_no)+'.jpg')
    time.sleep(1)

    if cnt == file_no :
        break

    f = open(ft_name, 'a',encoding='UTF-8')
    f.write("-----------------------------------------------------"+"\n")

    # 판매순위
    print("-" *70)
    ranking = li.find('p').get_text()
    
    print("1.판매순위:",ranking)
    f.write('1.판매순위:'+ ranking + "\n")

    #제품 소개 
    try :
        title1 = li.find('a',class_='itemname').get_text().replace("\n","")
    except AttributeError :
        title1 = ''
        print(title1.replace("\n",""))
        f.write('2.제품소개:'+ title1 + "\n")
    else :
        title2=title1.translate(bmp_map).replace("\n","") 
        print("2.제품소개:", title2.replace("\n",""))

        count += 1

        f.write('2.제품소개:'+ title2 + "\n")
            
    # 원래가격
    try :
        price = li.find('div','item_price').find('div','o-price').find('span').get_text().replace(",","")
    except AttributeError :
        price = ''
               
    print("3.원래가격:", price.replace(",",""))
    f.write('3.원래가격:'+ price + "\n")
    
    #판매가격
    try :
        sale_price = li.find('div','item_price').find('div','s-price').find('span').get_text().replace(",","")
    except (IndexError , AttributeError) :
        sale_price = '0'
        print('4.판매가격: ',sale_price)
        f.write('4.판매가격:'+ sale_price + "\n")
    else :
        print('4.판매가격:',sale_price)
        f.write('4.판매가격:'+ sale_price + "\n")

    #할인율
    try :
        rate_price = li.find('div','item_price').find('div','s-price').find('em').get_text()
    except AttributeError :
        rate_price=' '
               
    print('5.할인율:',rate_price)
    f.write('5.할인율:'+ rate_price + "\n")

    print("-" *70)

    f.close( )

    time.sleep(0.3)
        
    ranking2.append(ranking)
    title3.append(title2.replace("\n",""))
    price2.append(price.replace("\n",""))

    try :   
        sale_price2.append(sale_price)
    except IndexError :
        sale_price2.append(0)

    rate_price2.append(rate_price)

    if count == cnt :
        break

gmarket_best_seller = pd.DataFrame()
gmarket_best_seller['판매순위']=ranking2
gmarket_best_seller['제품소개']=pd.Series(title3)
gmarket_best_seller['원래가격']=pd.Series(price2)
gmarket_best_seller['판매가격']=pd.Series(sale_price2)
gmarket_best_seller['할인율']=pd.Series(rate_price2)


# csv 형태로 저장
gmarket_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장
gmarket_best_seller.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링정보 저장
orig_stdout = sys.stdout
f = open(ft_name, 'a',encoding='UTF-8')
sys.stdout = f


#이미지 삽입하기

import win32com.client as win32   #win32com모듈 사용, 윈도우 환경에서 사용할 수 있다고 함,,
import win32api  

excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet #엑셀 파일 열기
sheet.Columns(3).ColumnWidth = 30   #  이미지 넣을 칼럼 가로 사이즈 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 넣을 칼럼 세로 사이즈 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
    col_name='C'+str(a)
    col_name2.append(col_name)

for b in range(1,cnt+1) :
    file_name=img_dir+'\\'+str(b)+'.jpg'
    file_name2.append(file_name)
      
for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
    excel.Visible=True
    excel.ActiveWorkbook.Save()


print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

# 판매순위
# 제품소개
# 원래가격
# 판매가격
# 할인율

    지마켓 Best Seller 상품 정보 추출하기


   1.크롤링 할 건수는 몇건입니까?: 10
   2.결과 파일을 저장할 폴더명만 쓰세요(예:C:\py_temp\): C:\py_temp\
----------------------------------------------------------------------
1.판매순위: 1
2.제품소개: [바다원]진미채 1kg 국내가공 오징어채
3.원래가격: 29100원
4.판매가격: 16000원
5.할인율: 45%
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 2
2.제품소개: [크리넥스]3겹 순수소프트 화장지 27M 30롤X2팩/휴지 /new
3.원래가격: 42300원
4.판매가격: 31500원
5.할인율: 25%
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 3
2.제품소개: 쇼인더룸 15%쿠폰특가 여름신상 티셔츠/팬츠/빅사이즈
3.원래가격: 42900원
4.판매가격: 12900원
5.할인율: 69%
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 4
2.제품소개: [네파키즈](신세계타임스퀘어점패션관)[네파키즈]21SS 여름철 햇빛차단에 좋은 초경량 차양햇 정상가:49 000원
3.원래가격: 
4.판매가격: 34300원
5.할인율:  
--------

<ipython-input-1-1b9232b5e169>:185: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  gmarket_best_seller.to_excel(fx_name ,index=True)
